In [1]:
import torch
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from data_processing import create_interaction_matrix, create_adj_matrix
from model import LightGCN
from training import train_lightgcn
from recommend import get_recommendations

def create_sample_data():
    """Create sample interaction data"""
    # Create sample users and posts
    users = [
        "did:plc:user1",
        "did:plc:user2",
        "did:plc:user3",
        "did:plc:user4"
    ]
    
    posts = [
        "at://did:plc:creator1/app.bsky.feed.post/post1",
        "at://did:plc:creator1/app.bsky.feed.post/post2",
        "at://did:plc:creator2/app.bsky.feed.post/post3",
        "at://did:plc:creator2/app.bsky.feed.post/post4",
        "at://did:plc:creator3/app.bsky.feed.post/post5"
    ]
    
    # Create sample interactions (who liked what)
    interactions = [
        # user1 likes posts 1, 2, and 3
        {"user_uri": users[0], "post_uri": posts[0]},
        {"user_uri": users[0], "post_uri": posts[1]},
        {"user_uri": users[0], "post_uri": posts[2]},
        
        # user2 likes posts 2 and 4
        {"user_uri": users[1], "post_uri": posts[1]},
        {"user_uri": users[1], "post_uri": posts[3]},
        
        # user3 likes posts 1, 3, and 5
        {"user_uri": users[2], "post_uri": posts[0]},
        {"user_uri": users[2], "post_uri": posts[2]},
        {"user_uri": users[2], "post_uri": posts[4]},
        
        # user4 likes posts 4 and 5
        {"user_uri": users[3], "post_uri": posts[3]},
        {"user_uri": users[3], "post_uri": posts[4]},
    ]
    
    return pd.DataFrame(interactions)

def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Create and display sample data
    print("Creating sample data...")
    likes_df = create_sample_data()
    print("\nSample interactions:")
    print(likes_df)
    
    # Create interaction matrix
    print("\nCreating interaction matrix...")
    interaction_matrix, user_mapping, post_mapping = create_interaction_matrix(likes_df)
    
    print("\nUser mapping (DID -> index):")
    print(user_mapping)
    print("\nPost mapping (URI -> index):")
    print(post_mapping)
    
    print("\nInteraction matrix shape:", interaction_matrix.shape)
    print("Interaction matrix (dense format for visualization):")
    print(interaction_matrix.toarray())
    
    # Create adjacency matrix
    print("\nCreating adjacency matrix...")
    adj_matrix = create_adj_matrix(interaction_matrix)
    print("Adjacency matrix shape:", adj_matrix.size())
    print("Adjacency matrix (dense format for visualization):")
    print(adj_matrix.to_dense().numpy())
    
    # Initialize model
    print("\nInitializing model...")
    model = LightGCN(
        num_users=len(user_mapping),
        num_items=len(post_mapping),
        embedding_dim=4,  # Small embedding dim for example
        num_layers=2
    )
    
    # Training setup
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    # Train model
    print("\nTraining model...")
    train_lightgcn(
        model=model,
        optimizer=optimizer,
        train_data=interaction_matrix,
        adj_matrix=adj_matrix,
        epochs=5,  # Small number of epochs for example
        batch_size=4,
        device=device
    )
    
    # Get recommendations for first user
    print("\nGetting recommendations...")
    user_id = list(user_mapping.keys())[0]
    recommendations = get_recommendations(
        model=model,
        user_id=user_id,
        user_mapping=user_mapping,
        post_mapping=post_mapping,
        adj_matrix=adj_matrix,
        top_k=3,
        device=device
    )
    
    print(f"\nTop 3 recommendations for user {user_id}:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")

if __name__ == "__main__":
    main()

Creating sample data...

Sample interactions:
        user_uri                                        post_uri
0  did:plc:user1  at://did:plc:creator1/app.bsky.feed.post/post1
1  did:plc:user1  at://did:plc:creator1/app.bsky.feed.post/post2
2  did:plc:user1  at://did:plc:creator2/app.bsky.feed.post/post3
3  did:plc:user2  at://did:plc:creator1/app.bsky.feed.post/post2
4  did:plc:user2  at://did:plc:creator2/app.bsky.feed.post/post4
5  did:plc:user3  at://did:plc:creator1/app.bsky.feed.post/post1
6  did:plc:user3  at://did:plc:creator2/app.bsky.feed.post/post3
7  did:plc:user3  at://did:plc:creator3/app.bsky.feed.post/post5
8  did:plc:user4  at://did:plc:creator2/app.bsky.feed.post/post4
9  did:plc:user4  at://did:plc:creator3/app.bsky.feed.post/post5

Creating interaction matrix...

User mapping (DID -> index):
{'did:plc:user1': 0, 'did:plc:user2': 1, 'did:plc:user3': 2, 'did:plc:user4': 3}

Post mapping (URI -> index):
{'at://did:plc:creator1/app.bsky.feed.post/post1': 0, 'at://did:pl

c:\Users\Aleck\DSC180\Bluesky-GraphRec\lightgcn\data_processing.py:79: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:643.)
  adj = torch.sparse.FloatTensor(



Training model...
Epoch 0: Loss = 0.3192

Getting recommendations...

Top 3 recommendations for user did:plc:user1:
1. ('at://did:plc:creator1/app.bsky.feed.post/post1', 'https://bsky.app/profile/did:plc:creator1/post/post1')
2. ('at://did:plc:creator2/app.bsky.feed.post/post3', 'https://bsky.app/profile/did:plc:creator2/post/post3')
3. ('at://did:plc:creator1/app.bsky.feed.post/post2', 'https://bsky.app/profile/did:plc:creator1/post/post2')
